<a href="https://colab.research.google.com/github/JakobSchauser/BachelorProject-IceCube-ML/blob/main/Third_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install spektral -q

In [43]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader, Dataset
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GCNConv

from tqdm.notebook import tqdm

In [44]:
################################################################################
# LOAD DATA
################################################################################

class dat(Dataset):
  def __init__(self,n = 1,**kwargs):
    self.n = n
    super().__init__(**kwargs)
  def read(self):
    # path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/generatedDataEnergy100000 0.npz"
    path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/data.dat"
    dataset = np.load(path,allow_pickle = True)#["arr_0"]
    graphs = []
    for g in dataset:
      graphs.append(g)
    return np.array(dataset)

dataset = dat()

print("Dataset is", dataset, "consisting of", dataset[0])



#### Put on GPU when possible
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("Running on GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("Running on CPU")




Dataset is dat(n_graphs=100000) consisting of Graph(n_nodes=34, n_node_features=5, n_edge_features=None, n_labels=7)
Running on GPU


In [52]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 200  # Number of training epochs
batch_size = 32  # Batch size

network_size = 32 # one-variable network size changer


# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

train_loader = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
test_loader = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

In [54]:
################################################################################
# BUILD MODEL
################################################################################
X_in = Input(shape=(F,), name="X_in")
A_in = Input(shape=(None,), sparse=True, name="A_in")
# E_in = Input(shape=(S,), name="E_in")
I_in = Input(shape=(), name="segment_ids_in", dtype=tf.int32)

X_1 = GCNConv(32, activation="relu")([X_in, A_in])
X_2 = GCNConv(32, activation="relu")([X_1, A_in])
X_3 = GlobalSumPool()([X_2, I_in])
output = Dense(n_out)(X_3)

# Build model
model = Model(inputs=[X_in, A_in, I_in], outputs=output)
opt = Adam(lr=learning_rate)
loss_fn = MeanSquaredError()
acc_fn = MeanAbsoluteError()
model.compile()
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_in (InputLayer)               [(None, 5)]          0                                            
__________________________________________________________________________________________________
A_in (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
gcn_conv_12 (GCNConv)           (None, 32)           160         X_in[0][0]                       
                                                                 A_in[0][0]                       
__________________________________________________________________________________________________
gcn_conv_13 (GCNConv)           (None, 32)           1024        gcn_conv_12[0][0]          

In [55]:
################################################################################
# FIT MODEL
################################################################################
@tf.function(input_signature=train_loader.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions)
        acc = acc_fn(target,predictions)
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc


 
print("Fitting model")
current_batch = 0
model_loss = 0
model_acc = 0

epoch = 0

epoch_steps = train_loader.steps_per_epoch



t = tqdm(train_loader,initial = 0,total = train_loader.epochs*epoch_steps)
pb = []
tar = []
for batch in t:
    loss, acc = train_step(*batch)

    model_loss += loss
    model_acc += acc
    current_batch += 1
    t.set_description(f'Currently on epoch {epoch} of {epochs}')
    if current_batch == epoch_steps:
        s = "Loss: {} - Accuracy: {} - Epoch: {}".format(model_loss/epoch_steps, model_acc/epoch_steps, epoch)
        t.write(s)
        epoch += 1
        if epoch%2 == 0:
          print("Two current guesses are\n",np.array(pb[0][:3]),np.array(pb[1][:3]),"for\n",tar[0][:3],tar[1][:3])
        else:
          inp, tar = batch
          pb = model(inp, training=False)
        model_loss = 0
        model_acc = 0
        current_batch = 0
      



Fitting model


Loss: 42899574784.0 - Accuracy: 94144.453125 - Epoch: 0
Loss: 132292184.0 - Accuracy: 20378.873046875 - Epoch: 1
Two current guesses are
 [ -7138.813  -17340.89     6916.2397] [-52304.062 -45185.734 -19653.541] for
 [  1.86599042 -90.87925819 -45.60904589] [  2.29320716  86.58978536 -12.01520798]
Loss: 620779712.0 - Accuracy: 14617.0439453125 - Epoch: 2
Loss: 237627312.0 - Accuracy: 11748.56640625 - Epoch: 3
Two current guesses are
 [-1192.8009  -828.8773  2800.093 ] [-230.14467 -270.59604 2181.265  ] for
 [  0.84544047 -21.33659651 -88.04220241] [  2.3180051  174.69502847 -18.46663626]
Loss: 453761984.0 - Accuracy: 10120.392578125 - Epoch: 4
Loss: 111425520.0 - Accuracy: 9178.921875 - Epoch: 5
Two current guesses are
 [  625.0579   -109.66494 -1371.9594 ] [-257.16083  219.64757  846.4469 ] for
 [  1.48613198  94.98485903 -71.51337071] [  1.45300105 163.9399796  -46.78458392]
Loss: 122749304.0 - Accuracy: 8409.705078125 - Epoch: 6
Loss: 181599888.0 - Accuracy: 7832.7705078125 - Epoch: 

KeyboardInterrupt: ignored

In [56]:
model.save("1SavedModel-All")

# dataset targets = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]

<tf.Tensor: shape=(16, 7), dtype=float32, numpy=
array([[ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.6433990e+01, -4.5375439e+01,
        -1.3514052e-02,  3.3443925e-05,  7.9475202e-02],
       [ 1.5005592e+00,  3.0584778e+01, -2.64339

In [ ]:
################################################################################
# EVALUATE MODEL
################################################################################
print("Testing model")
model_loss = 0
model_acc = 0
steps = 0
test_loader = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

guesses, diffs, energies = [],[],[]
for batch in test_loader:
  steps += 1
  inputs, target = batch
  predictions = model(inputs, training=False)
  guesses.append(predictions)
  diffs.append(abs(predictions-target))
  energies.append(target)
  model_loss += loss_fn(target, predictions)
  model_acc += acc_fn(target, predictions)
print("An example for guesses is:")
print(guesses[0][:5],"for",energies[0][:5])



print("Done! \nTest loss: {}\nTest Acc: {}".format(model_loss/steps, model_acc/steps))



In [ ]:
import matplotlib.pyplot as plt
diffs = np.array(diffs).flatten()
energies = np.array(energies).flatten()
td,te = [],[]
for e in range(len(energies)):
    for a in range(len(energies[e])):
      td.append(diffs[e][a])
      te.append(energies[e][a])

plt.plot(te,td,'.')

In [ ]:
diffs[0][1]